In [1]:
from glob import glob
import os
import pickle
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Чтение данных в датафрейм:

In [2]:
PATH_TO_TEST_DATA = './test/'
PATH_TO_TRAIN_DATA = './train/'

In [3]:
def prepare_train_set(path_to_files):
    df = pd.DataFrame(columns=['text', 'target'])
    positive_files = glob(path_to_files + "pos/*.txt")
    negative_files = glob(path_to_files + "neg/*.txt")
    
    for file in tqdm(positive_files):
        with open(file, 'r', encoding="utf8") as content_file:
            content = content_file.read()
            df.loc[df.shape[0]] = [content, 1]
    for file in tqdm(negative_files):
        with open(file, 'r', encoding="utf8") as content_file:
            content = content_file.read()
            df.loc[df.shape[0]] = [content, 0]
    
    return df

In [4]:
df = prepare_train_set(PATH_TO_TRAIN_DATA)

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

Сериализуем датафрейм, чтобы не пересчитывать при перезапуске ядра.

In [5]:
with open ("./pickle/df.pkl", 'wb') as df_pkl:
    pickle.dump(df, df_pkl, protocol=2)

In [6]:
df = pickle.load(open("./pickle/df.pkl", "rb"))

In [10]:
pd.set_option('display.max_colwidth', -1)
df.head(3)

,text,target
0,bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt,1
1,homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter most people think of the homeless as just a lost cause while worrying about things such as racism the war on iraq pressuring kids to succeed technology the elections inflation or worrying if theyll be next to end up on the streetsbr br but what if you were given a bet to live on the streets for a month without the luxuries you once had from a home the entertainment sets a bathroom pictures on the wall a computer and everything you once treasure to see what its like to be homeless that is goddard bolts lessonbr br mel brooks who directs who stars as bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival jeffery tambor to see if he can live in the streets for thirty days without the luxuries if bolt succeeds he can do what he wants with a future project of making more buildings the bets on where bolt is thrown on the street with a bracelet on his leg to monitor his every move where he cant step off the sidewalk hes given the nickname pepto by a vagrant after its written on his forehead where bolt meets other characters including a woman by the name of molly lesley ann warren an exdancer who got divorce before losing her home and her pals sailor howard morris and fumes teddy wilson who are already used to the streets theyre survivors bolt isnt hes not used to reaching mutual agreements like he once did when being rich where its fight or flight kill or be killedbr br while the love connection between molly and bolt wasnt necessary to plot i found life stinks to be one of mel brooks observant films where prior to being a comedy it shows a tender side compared to his slapstick work such as blazing saddles young frankenstein or spaceballs for the matter to show what its like having something valuable before losing it the next day or on the other hand making a stupid bet like all rich people do when they dont know what to do with their money maybe they should give it to the homeless instead of using it like monopoly moneybr br or maybe this film will inspire you to help others,1
2,brilliant overacting by lesley ann warren best dramatic hobo lady i have ever seen and love scenes in clothes warehouse are second to none the corn on face is a classic as good as anything in blazing saddles the take on lawyers is also superb after being accused of being a turncoat selling out his boss and being dishonest the lawyer of pepto bolt shrugs indifferently im a lawyer he says three funny words jeffrey tambor a favorite from the later larry sanders show is fantastic here too as a mad millionaire who wants to crush the ghetto his character is more malevolent than usual the hospital scene and the scene where the homeless invade a demolition site are alltime classics look for the legs scene and the two big diggers fighting one bleeds this movie gets better each time i see it which is quite often,1


In [11]:
import string
df.text = df.text.apply(lambda x: x.lower())

tr = str.maketrans("", "", string.punctuation)
df.text = df.text.apply(lambda x: x.translate(tr))
df.text = df.text.str.replace('\d+', '')


In [12]:
df.head(1)

,text,target
0,bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt,1


Убираем стоп слова

In [13]:
from nltk.corpus import stopwords

stoplist = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stoplist]))

In [14]:
df.head(3)

,text,target
0,bromwell high cartoon comedy ran time programs school life teachers years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt,1
1,homelessness houselessness george carlin stated issue years never plan help street considered human everything going school work vote matter people think homeless lost cause worrying things racism war iraq pressuring kids succeed technology elections inflation worrying theyll next end streetsbr br given bet live streets month without luxuries home entertainment sets bathroom pictures wall computer everything treasure see like homeless goddard bolts lessonbr br mel brooks directs stars bolt plays rich man everything world deciding make bet sissy rival jeffery tambor see live streets thirty days without luxuries bolt succeeds wants future project making buildings bets bolt thrown street bracelet leg monitor every move cant step sidewalk hes given nickname pepto vagrant written forehead bolt meets characters including woman name molly lesley ann warren exdancer got divorce losing home pals sailor howard morris fumes teddy wilson already used streets theyre survivors bolt isnt hes used reaching mutual agreements like rich fight flight kill killedbr br love connection molly bolt wasnt necessary plot found life stinks one mel brooks observant films prior comedy shows tender side compared slapstick work blazing saddles young frankenstein spaceballs matter show like something valuable losing next day hand making stupid bet like rich people dont know money maybe give homeless instead using like monopoly moneybr br maybe film inspire help others,1
2,brilliant overacting lesley ann warren best dramatic hobo lady ever seen love scenes clothes warehouse second none corn face classic good anything blazing saddles take lawyers also superb accused turncoat selling boss dishonest lawyer pepto bolt shrugs indifferently im lawyer says three funny words jeffrey tambor favorite later larry sanders show fantastic mad millionaire wants crush ghetto character malevolent usual hospital scene scene homeless invade demolition site alltime classics look legs scene two big diggers fighting one bleeds movie gets better time see quite often,1


Применим PorterStemer на входные данные.

In [15]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def stem_sentence(sentence):
    porter_stemmer = PorterStemmer()
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['stem_text'] = df['text'].apply(stem_sentence)

Также применим лемматизацию в виде отдельного признака

In [16]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

def get_pos(word):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = sentence.split()
    lemmed_tokens = [lemmatizer.lemmatize(token, get_pos(token)) for token in tokens]
    return ' '.join(lemmed_tokens)

df['lemm_text'] = df['text'].apply(lemmatize_sentence)

In [17]:
df.head(1)

,text,target,stem_text,lemm_text
0,bromwell high cartoon comedy ran time programs school life teachers years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt,1,bromwel high cartoon comedi ran time program school life teacher year teach profess lead believ bromwel high satir much closer realiti teacher scrambl surviv financi insight student see right pathet teacher pomp petti whole situat remind school knew student saw episod student repeatedli tri burn school immedi recal high classic line inspector im sack one teacher student welcom bromwel high expect mani adult age think bromwel high far fetch piti isnt,bromwell high cartoon comedy run time program school life teacher year teaching profession lead believe bromwell high satire much close reality teacher scramble survive financially insightful student see right pathetic teacher pomp pettiness whole situation remind school know student saw episode student repeatedly try burn school immediately recall high classic line inspector im sack one teacher student welcome bromwell high expect many adult age think bromwell high far fetch pity isnt


Применим аналогичные операции для отложенной выборки:

In [18]:
test_df = prepare_train_set(PATH_TO_TEST_DATA)
test_df.text = test_df.text.apply(lambda x: x.lower())
test_df.text = test_df.text.apply(lambda x: x.translate(tr))
test_df.text = test_df.text.str.replace('\d+', '')
test_df.text = test_df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stoplist]))
test_df['stem_text'] = test_df['text'].apply(stem_sentence)
test_df['lemm_text'] = test_df['text'].apply(lemmatize_sentence)

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

Применим bag-of-words для получения фич по токенам:

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
import operator
import warnings
warnings.filterwarnings('ignore')

In [20]:
def get_top_words(vectorizer, model, n = 10):
    coefs_dict = dict(zip(vectorizer.get_feature_names(), model.coef_[0]))
    sorted_dict = sorted(coefs_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict[:n]

def get_negative_words(vectorizer, model, n = 10):
    coefs_dict = dict(zip(vectorizer.get_feature_names(), model.coef_[0]))
    sorted_dict = sorted(coefs_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict[-n:]

def train_and_predict_bow(estimator, count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='text'):
    print("Model info: " + estimator.__class__.__name__ + " on dataset " + column + " preprocessed with " 
          + count_vectorizer.__class__.__name__)
    
    X_train = df[column]
    y_train = df['target']
    y_train = y_train.astype('bool')
    X_train_bow = count_vectorizer.fit_transform(X_train)
    estimator.fit(X_train_bow, y_train)  
    
    X_test_bow = count_vectorizer.transform(test_df[column])
    y_test = test_df.target
    y_test=y_test.astype('bool')


    print("accuracy: " + str(accuracy_score(y_test, estimator.predict(X_test_bow))))
    print("roc auc: " + str(roc_auc_score(y_test, estimator.predict(X_test_bow))))
    print('positive: ')
    print(get_top_words(count_vectorizer, estimator))
    print('negative: ')
    print(get_negative_words(count_vectorizer, estimator))
    print("\n")    

Начнём со стохастического градиентного спуска. Попробуем модель на:
- простом тексте без стоп слов и знаков препинаний;
- на нём же после стемминга;
- на нем же после лемматизации;

In [21]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text')
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text')
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text')

Model info: SGDClassifier on dataset text preprocessed with CountVectorizer
accuracy: 0.83924
roc auc: 0.83924
positive: 
[('refreshing', 4.127016881086339), ('subtle', 3.7301883348280396), ('vengeance', 3.4920912070730625), ('flawless', 3.412725497821397), ('enjoyable', 3.412725497821396), ('rare', 3.1746283700664324), ('perfectly', 3.1746283700664293), ('spectacular', 3.095262660814763), ('wonderfully', 3.0952626608147593), ('complain', 3.095262660814756)]
negative: 
[('horrible', -3.7301883348280436), ('disappointing', -3.888919753331373), ('mstk', -3.968285462583027), ('awful', -3.9682854625830304), ('forgettable', -4.365114008841333), ('lacks', -4.365114008841333), ('poorly', -4.761942555099629), ('waste', -5.1587711013579405), ('worst', -5.317502519861252), ('disappointment', -5.55559964761624)]


Model info: SGDClassifier on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.83804
roc auc: 0.83804
positive: 
[('refresh', 5.23813681060958), ('flawless', 4.04765117183

Как ни странно, стемминг дал чуть лучший результат чем лемматизация, последняя в свою очередь лучше простого текста. Правда, тут видна некоторая утечка данных - такие слова как '710', '810', '1010'  есть ни что иное, как оценки пользователей без символа '/'. В принципе их вполне можно считать как признаки, но для вычисления именно значящих слов, после первого теста моделей создадим ещё один вариант входных значений без цифр и чисел.

Добавим n-gram = 2

In [22]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))

Model info: SGDClassifier on dataset text preprocessed with CountVectorizer
accuracy: 0.84372
roc auc: 0.8437199999999999
positive: 
[('refreshing', 4.444479718092982), ('well worth', 4.047651171834684), ('wonderfully', 3.8889197533313684), ('funniest', 3.888919753331367), ('must see', 3.5714569163247245), ('rare', 3.571456916324721), ('today', 3.492091207073062), ('superb', 3.412725497821396), ('flawless', 3.333359788569741), ('vengeance', 3.1746283700664204)]
negative: 
[('horrible', -3.809554044079705), ('lousy', -3.8095540440797064), ('weak', -3.809554044079711), ('forgettable', -4.206382590338005), ('baldwin', -4.365114008841326), ('worst', -4.523845427344644), ('awful', -4.603211136596322), ('waste', -4.920673973602949), ('poorly', -5.000039682854611), ('disappointment', -5.7936967753712025)]


Model info: SGDClassifier on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.84248
roc auc: 0.8424800000000001
positive: 
[('well worth', 4.206382590338015), ('funniest', 3

На биграммах лучше показал себя простой текст.

In [23]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,3)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,3)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,3)))

Model info: SGDClassifier on dataset text preprocessed with CountVectorizer
accuracy: 0.8326
roc auc: 0.8326
positive: 
[('vengeance', 3.888919753331351), ('wonderfully', 3.8095540440797087), ('refreshing', 3.7301883348280453), ('well worth', 3.7301883348280405), ('funniest', 3.571456916324718), ('surprisingly good', 3.4920912070730608), ('mildred', 3.2539940793180846), ('surprisingly', 3.2539940793180793), ('alexandre', 3.2539940793180744), ('incredible', 3.253994079318073)]
negative: 
[('worst', -3.6508226255763927), ('worse', -3.7301883348280453), ('lacks', -3.7301883348280485), ('laughable', -3.8095540440797055), ('awful', -3.8095540440797073), ('horrible', -3.8889197533313555), ('lousy', -3.968285462583027), ('waste', -4.365114008841337), ('poorly', -4.920673973602952), ('disappointment', -5.952428193874534)]


Model info: SGDClassifier on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.80624
roc auc: 0.8062399999999998
positive: 
[('favorit', 4.44447971809299), ('

Триграммы не принесли новых значимых фич в топ-20, оставим для следующих моделей максимум в 2.

Попробуем байесовский классификатор:

In [24]:
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='stem_text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='lemm_text')

Model info: MultinomialNB on dataset text preprocessed with CountVectorizer
accuracy: 0.8364
roc auc: 0.8364
positive: 
[('br', -3.8972706078900003), ('film', -4.248481392850371), ('movie', -4.325564896712059), ('one', -4.65356390157368), ('like', -5.056850983945498), ('good', -5.222750243966832), ('story', -5.354159763146175), ('great', -5.384204269880783), ('time', -5.431034481509069), ('see', -5.458202270688039)]
negative: 
[('delia', -14.13171521635923), ('hobgoblins', -14.13171521635923), ('kareena', -14.13171521635923), ('kornbluth', -14.13171521635923), ('saif', -14.13171521635923), ('sarne', -14.13171521635923), ('slater', -14.13171521635923), ('tashan', -14.13171521635923), ('tremors', -14.13171521635923), ('welch', -14.13171521635923)]


Model info: MultinomialNB on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.82156
roc auc: 0.8215600000000001
positive: 
[('br', -3.9593016801438665), ('film', -4.091202980291849), ('movi', -4.204081039835916), ('one', -4.668

In [25]:
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)), column='text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)), column='stem_text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)), column='lemm_text')

Model info: MultinomialNB on dataset text preprocessed with CountVectorizer
accuracy: 0.84704
roc auc: 0.84704
positive: 
[('br', -3.9573191818423794), ('film', -4.30852996680275), ('movie', -4.385613470664438), ('one', -4.713612475526059), ('like', -5.116899557897877), ('good', -5.282798817919211), ('story', -5.414208337098554), ('great', -5.444252843833162), ('time', -5.4910830554614485), ('see', -5.518250844640418)]
negative: 
[('thunderbirds', -13.498616609751664), ('total waste', -13.498616609751664), ('uwe', -13.498616609751664), ('uwe boll', -13.498616609751664), ('wayans', -13.498616609751664), ('worst films', -13.498616609751664), ('acting horrible', -14.19176379031161), ('hours life', -14.19176379031161), ('prom night', -14.19176379031161), ('worst acting', -14.19176379031161)]


Model info: MultinomialNB on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.84172
roc auc: 0.8417200000000001
positive: 
[('br', -4.044338223920716), ('film', -4.1762395240686985), (

Тут мы видим довольно много распространенных слов, но это происходит во многом из-за их высокой априорной вероятности. В негативных значения поинтереснее(особенно uwe и uwe boll, а также wayans - плохие режиссеры становятся именем нарицательным).

In [26]:
train_and_predict_bow(estimator=svm.LinearSVC(), column='text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='stem_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='lemm_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))

Model info: LinearSVC on dataset text preprocessed with CountVectorizer
accuracy: 0.83348
roc auc: 0.83348
positive: 
[('flawless', 1.1552537763145996), ('well worth', 1.1392147209892094), ('br must', 1.1192352433061774), ('refreshing', 1.0831790902172522), ('surprisingly good', 1.0509433682998752), ('pickup', 1.0383345022552855), ('hoot', 1.0250721088028092), ('even better', 1.007444100540922), ('marvelous', 0.9829519273581528), ('able see', 0.9736849264754289)]
negative: 
[('lousy', -0.9931604127891982), ('one worst', -0.9951447902897348), ('lifeless', -0.997737390387638), ('dreadful', -1.043698711823755), ('hours life', -1.0588304139935838), ('poorly', -1.0924852184736424), ('forgettable', -1.1311505132453612), ('mstk', -1.148915233089379), ('disappointment', -1.1737277182011188), ('baldwin', -1.238212324376126)]


Model info: LinearSVC on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.82736
roc auc: 0.82736
positive: 
[('never bore', 1.2544241758736523), ('driven',

Повторим все те же операции, но теперь с TF-IDF вместо Bag-of-Words. Будем использовать сразу биграммы.

In [27]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))

train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='stem_text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='lemm_text')

train_and_predict_bow(estimator=svm.LinearSVC(), column='text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='stem_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='lemm_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))

Model info: SGDClassifier on dataset text preprocessed with TfidfVectorizer
accuracy: 0.88528
roc auc: 0.8852800000000001
positive: 
[('excellent', 3.6851802375723444), ('great', 3.622783375328195), ('perfect', 2.8668162303378333), ('best', 2.7884322364604253), ('wonderful', 2.711613125191703), ('amazing', 2.617981408422977), ('favorite', 2.5358047435466973), ('fun', 2.4392112748273194), ('today', 2.180208917040142), ('brilliant', 2.1556243691031556)]
negative: 
[('unfortunately', -2.83793542494793), ('disappointment', -2.898411813104424), ('terrible', -2.9223978653043434), ('nothing', -3.065593966991999), ('waste', -3.453917075793234), ('poor', -3.476058421171242), ('boring', -3.4968406384410264), ('awful', -3.935910204335568), ('bad', -4.010650715468367), ('worst', -4.9618553995989565)]


Model info: SGDClassifier on dataset stem_text preprocessed with TfidfVectorizer
accuracy: 0.88292
roc auc: 0.8829199999999999
positive: 
[('excel', 3.951958068207169), ('great', 3.7921424196672313)